In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'
sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util  
import signature_attribution_util
import clonality_analysis_util

In [2]:
allImpactMuts = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,36,39,85,136) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
allImpactMuts = maf_analysis_utils.fix_mll_genes(allImpactMuts)
impact368genes = set(['ABL1', 'AKT1', 'AKT2', 'AKT3', 'ALK', 'ALOX12B', 'APC', 'AR', 'ARAF', 'ARID1A', 'ARID1B', 'ARID2', 'ARID5B', 'ASXL1', 'ASXL2', 'ATM', 'ATR', 'ATRX', 'AURKA', 'AURKB', 'AXIN1', 'AXIN2', 'AXL', 'B2M', 'BAP1', 'BARD1', 'BBC3', 'BCL2', 'BCL2L1', 'BCL2L11', 'BCL6', 'BCOR', 'BLM', 'BMPR1A', 'BRAF', 'BRCA1', 'BRCA2', 'BRD4', 'BRIP1', 'BTK', 'CARD11', 'CASP8', 'CBFB', 'CBL', 'CCND1', 'CCND2', 'CCND3', 'CCNE1', 'CD274', 'CD276', 'CD79B', 'CDC73', 'CDH1', 'CDK12', 'CDK4', 'CDK6', 'CDK8', 'CDKN1A', 'CDKN1B', 'CDKN2A', 'CDKN2B', 'CDKN2C', 'CHEK1', 'CHEK2', 'CIC', 'CREBBP', 'CRKL', 'CRLF2', 'CSF1R', 'CTCF', 'CTLA4', 'CTNNB1', 'CUL3', 'DAXX', 'DCUN1D1', 'DDR2', 'DICER1', 'DIS3', 'DNMT1', 'DNMT3A', 'DNMT3B', 'DOT1L', 'E2F3', 'EED', 'EGFL7', 'EGFR', 'EIF1AX', 'EP300', 'EPCAM', 'EPHA3', 'EPHA5', 'EPHB1', 'ERBB2', 'ERBB3', 'ERBB4', 'ERCC2', 'ERCC3', 'ERCC4', 'ERCC5', 'ERG', 'ESR1', 'ETV1', 'ETV6', 'EZH2', 'FAM123B', 'FAM175A', 'FAM46C', 'FANCA', 'FANCC', 'FAT1', 'FBXW7', 'FGF19', 'FGF3', 'FGF4', 'FGFR1', 'FGFR2', 'FGFR3', 'FGFR4', 'FH', 'FLCN', 'FLT1', 'FLT3', 'FLT4', 'FOXA1', 'FOXL2', 'FOXP1', 'FUBP1', 'GATA1', 'GATA2', 'GATA3', 'GNA11', 'GNAQ', 'GNAS', 'GREM1', 'GRIN2A', 'GSK3B', 'H3F3C', 'HGF', 'HIST1H1C', 'HIST1H2BD', 'HIST1H3B', 'HNF1A', 'HRAS', 'ICOSLG', 'IDH1', 'IDH2', 'IFNGR1', 'IGF1', 'IGF1R', 'IGF2', 'IKBKE', 'IKZF1', 'IL10', 'IL7R', 'INPP4A', 'INPP4B', 'INSR', 'IRF4', 'IRS1', 'IRS2', 'JAK1', 'JAK2', 'JAK3', 'JUN', 'KDM5A', 'KDM5C', 'KDM6A', 'KDR', 'KEAP1', 'KIT', 'KLF4', 'KRAS', 'LATS1', 'LATS2', 'LMO1', 'MAP2K1', 'MAP2K2', 'MAP2K4', 'MAP3K1', 'MAP3K13', 'MAPK1', 'MAX', 'MCL1', 'MDC1', 'MDM2', 'MDM4', 'MED12', 'MEF2B', 'MEN1', 'MET', 'MITF', 'MLH1', 'MLL', 'MLL2', 'MLL3', 'MPL', 'MRE11A', 'MSH2', 'MSH6', 'MTOR', 'MUTYH', 'MYC', 'MYCL1', 'MYCN', 'MYD88', 'MYOD1', 'NBN', 'NCOR1', 'NF1', 'NF2', 'NFE2L2', 'NKX2-1', 'NKX3-1', 'NOTCH1', 'NOTCH2', 'NOTCH3', 'NOTCH4', 'NPM1', 'NRAS', 'NSD1', 'NTRK1', 'NTRK2', 'NTRK3', 'PAK1', 'PAK7', 'PALB2', 'PARK2', 'PARP1', 'PAX5', 'PBRM1', 'PDCD1', 'PDGFRA', 'PDGFRB', 'PDPK1', 'PHOX2B', 'PIK3C2G', 'PIK3C3', 'PIK3CA', 'PIK3CB', 'PIK3CD', 'PIK3CG', 'PIK3R1', 'PIK3R2', 'PIK3R3', 'PIM1', 'PLK2', 'PMAIP1', 'PMS1', 'PMS2', 'PNRC1', 'POLE', 'PPP2R1A', 'PRDM1', 'PRKAR1A', 'PTCH1', 'PTEN', 'PTPN11', 'PTPRD', 'PTPRS', 'PTPRT', 'RAC1', 'RAD50', 'RAD51', 'RAD51B', 'RAD51C', 'RAD51D', 'RAD52', 'RAD54L', 'RAF1', 'RARA', 'RASA1', 'RB1', 'RBM10', 'RECQL4', 'REL', 'RET', 'RFWD2', 'RHOA', 'RICTOR', 'RIT1', 'RNF43', 'ROS1', 'RPS6KA4', 'RPS6KB2', 'RPTOR', 'RUNX1', 'RYBP', 'SDHA', 'SDHAF2', 'SDHB', 'SDHC', 'SDHD', 'SETD2', 'SF3B1', 'SH2D1A', 'SHQ1', 'SMAD2', 'SMAD3', 'SMAD4', 'SMARCA4', 'SMARCB1', 'SMARCD1', 'SMO', 'SOCS1', 'SOX17', 'SOX2', 'SOX9', 'SPEN', 'SPOP', 'SRC', 'STAG2', 'STK11', 'STK40', 'SUFU', 'SUZ12', 'SYK', 'TBX3', 'TERT', 'TET1', 'TET2', 'TGFBR1', 'TGFBR2', 'TMEM127', 'TMPRSS2', 'TNFAIP3', 'TNFRSF14', 'TOP1', 'TP53', 'TP63', 'TRAF7', 'TSC1', 'TSC2', 'TSHR', 'U2AF1', 'VHL', 'VTCN1', 'WT1', 'XIAP', 'XPO1', 'YAP1', 'YES1'])
#ONLY do analysis on impact 368 genes
allImpactMuts368 = allImpactMuts[allImpactMuts['Hugo_Symbol'].isin(impact368genes)]


In [4]:
allImpactMuts368 = maf_analysis_utils.fix_mll_genes(allImpactMuts368)

/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myUtils/maf_analysis_utils.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  maf['Hugo_Symbol'] = maf['Hugo_Symbol'].apply(lambda x:


In [4]:
###########

#nPossibleMuts

simulatedMutationDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/simulatedMutationSummary.tsv')
nOncogenicMutDict = dict(zip(simulatedMutationDf['Hugo_Symbol'], simulatedMutationDf['totalNOncogenic']))
nMutDict = dict(zip(simulatedMutationDf['Hugo_Symbol'], simulatedMutationDf['nPossibleMuts']))


In [5]:
endometrialHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'Hypermutated')
colorectalHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'Hypermutated')
gliomaHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'Hypermutated')

endoHyperDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(endometrialHyperIds)]
colorectalHyperDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(colorectalHyperIds)]
gliomaHyperDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(gliomaHyperIds)]


In [98]:
endometrialIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'all')
colorectalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'all')
gliomaIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'all')

endoDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(endometrialIds)]
colorectalDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(colorectalIds)]
gliomaDf = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(gliomaIds)]


In [99]:
#make sure the genes we look at only overlap
#JUST does filtering on gene names
endoHyperDf = endoHyperDf[endoHyperDf['Hugo_Symbol'].isin(nOncogenicMutDict.keys())]
colorectalHyperDf = colorectalHyperDf[colorectalHyperDf['Hugo_Symbol'].isin(nOncogenicMutDict.keys())]
gliomaHyperDf = gliomaHyperDf[gliomaHyperDf['Hugo_Symbol'].isin(nOncogenicMutDict.keys())]

endoDf = endoDf[endoDf['Hugo_Symbol'].isin(nOncogenicMutDict.keys())]
colorectalDf = colorectalDf[colorectalDf['Hugo_Symbol'].isin(nOncogenicMutDict.keys())]
gliomaDf = gliomaDf[gliomaDf['Hugo_Symbol'].isin(nOncogenicMutDict.keys())]


In [100]:
#endoRanking = maf_analysis_utils.enumerate_gene_mut_ranking_for_cohort(endoHyperDf)
#colorectalRanking = maf_analysis_utils.enumerate_gene_mut_ranking_for_cohort(colorectalHyperDf)
#gliomaRanking = maf_analysis_utils.enumerate_gene_mut_ranking_for_cohort(gliomaHyperDf)

#CHOOSE TO DO ON ALL OR HYPERMUTATEDs
endoOncRanking = maf_analysis_utils.enumerate_gene_mut_ranking_for_cohort(endoDf)
colorectalOncRanking = maf_analysis_utils.enumerate_gene_mut_ranking_for_cohort(colorectalDf)
gliomaOncRanking = maf_analysis_utils.enumerate_gene_mut_ranking_for_cohort(gliomaDf)


In [101]:
endoVUSRanking = maf_analysis_utils.enumerate_gene_mut_ranking_for_cohort(endoDf, mode='vus')
colorectalVUSRanking = maf_analysis_utils.enumerate_gene_mut_ranking_for_cohort(colorectalDf, mode='vus')
gliomaVUSRanking = maf_analysis_utils.enumerate_gene_mut_ranking_for_cohort(gliomaDf, mode='vus')


In [10]:
def normalize_ranking_by_n_mut_of_interest(unnormRankingDict, possibleMutD):
    
    def normalize_dict(d):
        factor=1.0/max(d.values())
        for k in d:
          d[k] = d[k]*factor
        return d
    
    #ordered list of genes
    newD = {}
    runningSum = 0
    for gene, rank in sorted(list(unnormRankingDict.items()), key = lambda x: x[1]):
    #WE IGNORE THE RANK and just make our new dictionary in the order of rankings spaced by possible mutations
        runningSum += possibleMutD[gene]
        newD[gene] = runningSum
    normedD = normalize_dict(newD)
    return normedD

In [102]:
#normedEndoOncD = normalize_ranking_by_n_mut_of_interest(endoOncRanking, nOncogenicMutDict)
#normedColoOncD = normalize_ranking_by_n_mut_of_interest(colorectalOncRanking, nOncogenicMutDict)
#normedGliomaOncD = normalize_ranking_by_n_mut_of_interest(gliomaOncRanking, nOncogenicMutDict)

normedEndoOncD = normalize_ranking_by_n_mut_of_interest(endoOncRanking, nMutDict)
normedColoOncD = normalize_ranking_by_n_mut_of_interest(colorectalOncRanking, nMutDict)
normedGliomaOncD = normalize_ranking_by_n_mut_of_interest(gliomaOncRanking, nMutDict)

In [103]:
normedEndoVUSD = normalize_ranking_by_n_mut_of_interest(endoVUSRanking, nMutDict)
normedColoVUSD = normalize_ranking_by_n_mut_of_interest(colorectalVUSRanking, nMutDict)
normedGliomaVUSD = normalize_ranking_by_n_mut_of_interest(gliomaVUSRanking, nMutDict)

In [ ]:
impactSigs = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/signatures_from_unfiltered_maf.txt')

In [13]:
#returns a dataframe with x, y values for the mutations in the case
def assign_x_y_values_for_cases(mutationDf, ranking, mode = 'oncogenic'):
    listOfDicts = []
    muts = mutationDf
    if mode == 'oncogenic':
        muts = mutationDf[mutationDf['oncogenic'].notnull()]
    elif mode == 'vus':
        muts = mutationDf[~mutationDf['oncogenic'].notnull()]
    print muts.shape
    for case in set(mutationDf['Tumor_Sample_Barcode']):
        caseMaf = muts[muts['Tumor_Sample_Barcode'] == case]
        mutatedGenes = list(caseMaf['Hugo_Symbol'])
        
        geneRankList = [(gene, ranking[gene]) for gene in mutatedGenes]
        sortedGeneRankList = sorted(geneRankList, key= lambda x: x[1])
        nMutatedGenes = len(sortedGeneRankList)
        
        #print '__________-', case
        #print nMutatedGenes
    
        prevY = 0
        cntr = 0.0
        inflectionHappened = False
        #We append an entry to the dataframe for the beginning
        listOfDicts.append({'Tumor_Sample_Barcode': case, 'd/dx': None, 
                            'Hugo_Symbol': None, 'x': 0, 'y': 0, 'n': cntr,
                           'inflection': False})
        for gene, rank in sortedGeneRankList:
            cntr += 1
            x = cntr/nMutatedGenes
            y = rank
            prevX = (cntr - 1)/nMutatedGenes
            dDx = (y - prevY)/(x - prevX)
            
            #print gene, dDx
            
            #mark the inflection
            inflection = False
            if not inflectionHappened and dDx > 1:
                inflectionHappened = True
                inflection = True
                
            listOfDicts.append({'Tumor_Sample_Barcode': case, 'd/dx': dDx, 
                            'Hugo_Symbol': gene, 'x': x, 'y': y, 'n': cntr,
                               'inflection': inflection})
            
            prevY = y
    return pd.DataFrame(listOfDicts)
        

        

In [17]:
def get_nmut_under_selection(cancerTypeMaf, oncD, vusD):
    cancerTypeGeneGiniOnc = assign_x_y_values_for_cases(cancerTypeMaf, oncD, mode = 'oncogenic')
    cancerTypeGeneGiniVus = assign_x_y_values_for_cases(cancerTypeMaf, vusD, mode = 'vus')
    
    #Get the number of mutations under selection
    inflectionDfOnc = cancerTypeGeneGiniOnc[cancerTypeGeneGiniOnc['inflection'] == True]
    nmutUnderSelectionDictOnc = dict(zip(inflectionDfOnc['Tumor_Sample_Barcode'], inflectionDfOnc['n']))
    inflectionDfVus = cancerTypeGeneGiniVus[cancerTypeGeneGiniVus['inflection'] == True]
    nmutUnderSelectionDictVus = dict(zip(inflectionDfVus['Tumor_Sample_Barcode'], inflectionDfVus['n']))

    return nmutUnderSelectionDictOnc, nmutUnderSelectionDictVus

In [54]:

#MAKE THE X AXIS N ONCOGENIC MUTATIONS IN CASE
def build_df_for_plotting(mutsMaf, sigsDf, nmutUnderSelectionOnc, nmutUnderSelectionVus):
    
    sigsDf['dominantSignature'] = sigsDf.apply(lambda row: 
        mutationSigUtils.get_dominant_signature(row.to_dict(), cols=None, prefix='mean', notEnoughMuts= True), axis=1)
    dominantSignatureDict = dict(zip(sigsDf['Tumor_Sample_Barcode'], sigsDf['dominantSignature']))
    nmutMbDict = dict(zip(sigsDf['Tumor_Sample_Barcode'], sigsDf['Nmut_Mb']))
    nmutOncDict = maf_analysis_utils.get_n_oncogenic_muts_per_case_dict(mutsMaf)
    
    listOfDicts = []
    for tsb, nmutS in nmutUnderSelectionOnc.items():
        nOnc = 0
        if tsb in nmutOncDict:
            nOnc = nmutOncDict[tsb]
        if tsb in nmutMbDict:
            listOfDicts.append({'Tumor_Sample_Barcode': tsb, 'nmutUnderSelection': nmutS,
                           'Nmut_Mb': nmutMbDict[tsb], 'nOnc': nOnc, 'type': 'Driver'})
        
    for tsb, nmutS in nmutUnderSelectionVus.items():
        nOnc = 0
        if tsb in nmutOncDict:
            nOnc = nmutOncDict[tsb]
        if tsb in nmutMbDict:
            listOfDicts.append({'Tumor_Sample_Barcode': tsb, 'nmutUnderSelection': nmutS,
                           'Nmut_Mb': nmutMbDict[tsb], 'nOnc': nOnc, 'type': 'VUS'})

    df = pd.DataFrame(listOfDicts)
    df['HypermutantStatus'] = df['Nmut_Mb'].apply(lambda x: 'Hypermutated' if x <= 30 else 'Not-Hypermutated')
    df['class_and_type'] = df['HypermutantStatus'] + '_' + df['type']
    #set signatures to display()
    df['dominantSignature'] = df['Tumor_Sample_Barcode'].apply(lambda x: dominantSignatureDict[x] if x in dominantSignatureDict else None)
    df['dominantSignature'] = df['dominantSignature'].apply(lambda x: x if x in set(['mean_10', 'mean_6', 'mean_14']) else 'other')
    return df
    
    

In [36]:
endometrialNmutUnderSelectionDictOnc, endometrialNmutUnderSelectionDictVus = get_nmut_under_selection(endoDf, normedEndoOncD, normedEndoVUSD)


(8993, 137)
(18032, 137)


In [64]:
dfEndo = build_df_for_plotting(endoDf, impactSigs, 
                           endometrialNmutUnderSelectionDictOnc, endometrialNmutUnderSelectionDictVus)

**tmp break**

In [58]:
colorectalNmutUnderSelectionDictOnc, colorectalNmutUnderSelectionDictVus = get_nmut_under_selection(colorectalDf, normedColoOncD, normedColoVUSD)


(17021, 137)
(24198, 137)


In [60]:
dfColo = build_df_for_plotting(colorectalDf, impactSigs, 
                           colorectalNmutUnderSelectionDictOnc, colorectalNmutUnderSelectionDictVus)

**Glioma**

In [104]:
gliomaNmutUnderSelectionDictOnc, gliomaNmutUnderSelectionDictVus = get_nmut_under_selection(gliomaDf, normedGliomaOncD, normedGliomaVUSD)


(4789, 137)
(7493, 137)


In [105]:
dfGlioma = build_df_for_plotting(gliomaDf, impactSigs, 
                           gliomaNmutUnderSelectionDictOnc, gliomaNmutUnderSelectionDictVus)

In [106]:
dfEndo['cancerType'] = 'Endometrial'
dfColo['cancerType'] = 'Colorectal'
dfGlioma['cancerType'] = 'Glioma'

In [107]:
df = pd.concat([dfEndo, dfColo, dfGlioma])

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [112]:
df['classAndCancerType'] = df['cancerType'] + '_' + df['type']

In [113]:
#give each class a marking based on whether they are hypermutant/indeterminate/neither
endometrialMutClassDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds/Endometrial_Cancer.tsv')
endometrialMutClassDict = dict(zip(endometrialMutClassDf['Tumor_Sample_Barcode'], endometrialMutClassDf['hypermutantClassification']))

colorectalMutClassDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds/Colorectal_Cancer.tsv')
colorectalMutClassDict = dict(zip(colorectalMutClassDf['Tumor_Sample_Barcode'], colorectalMutClassDf['hypermutantClassification']))

gliomaMutClassDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds/Glioma.tsv')
gliomaMutClassDict = dict(zip(gliomaMutClassDf['Tumor_Sample_Barcode'], gliomaMutClassDf['hypermutantClassification']))

In [115]:
#big lambda function
def set_case_class(row, endometrialMutClassDict, colorectalMutClassDict, gliomaMutClassDict):
    if row['cancerType'] == 'Endometrial':
        if row['Tumor_Sample_Barcode'] in endometrialMutClassDict:
            return 'Endometrial_' + endometrialMutClassDict[row['Tumor_Sample_Barcode']]
        else:
            return None
    elif row['cancerType'] == 'Colorectal':
        if row['Tumor_Sample_Barcode'] in colorectalMutClassDict:
            return 'Colorectal_' + colorectalMutClassDict[row['Tumor_Sample_Barcode']]
        else:
            return None
        
    elif row['cancerType'] == 'Glioma':
        if row['Tumor_Sample_Barcode'] in gliomaMutClassDict:
            return 'Glioma_' + gliomaMutClassDict[row['Tumor_Sample_Barcode']]
        else:
            return None
    else: 
        return None
        

df['caseClass'] = df.apply(lambda row:
    set_case_class(row, endometrialMutClassDict, colorectalMutClassDict, gliomaMutClassDict), axis=1)

In [122]:
df['displayGroup'] = df.apply(lambda row: row['caseClass'] if row['type'] == 'Driver' else 'VUS', axis=1)
#FOR SAKE OF CLARITY 
df = df[~df['displayGroup'].isin(['Colorectal_Indeterminate', 'Endometrial_Indeterminate', 'Glioma_Indeterminate'])]

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [123]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/nmutUnderSelection_endometrial.tsv', index=False, sep='\t')

In [121]:
Counter(df['displayGroup'])

Counter({'Colorectal_Hypermutated': 249,
         'Colorectal_Normal': 589,
         'Endometrial_Hypermutated': 216,
         'Endometrial_Indeterminate': 47,
         'Endometrial_Normal': 175,
         'Glioma_Hypermutated': 42,
         'Glioma_Normal': 81,
         'VUS': 2334})